In [1]:
import sys
sys.path.append('../packages')

import requests
import json
import uuid
from textservice import segmentedtext

Retrieve the annotation for a specific page in the sample TEI document

In [2]:
page_id = "page-21"
url = "http://localhost:5001/annotations/"+page_id
response = requests.get(url)

In [3]:
response.json()

{'annotations': {'begin_anchor': {'identifier': 'anchor_5182d4c7-d319-4c85-a09c-fa58e6ba0958',
   'sequence_number': 386},
  'end_anchor': {'identifier': 'anchor_959ec341-c694-4f7b-b040-258bfc191f2d',
   'sequence_number': 394},
  'id': 'page-21',
  'label': 'page',
  'resource_id': 'bosb002graa04_01'}}

In [4]:
page_annotation = response.json()['annotations']
page_annotation

{'begin_anchor': {'identifier': 'anchor_5182d4c7-d319-4c85-a09c-fa58e6ba0958',
  'sequence_number': 386},
 'end_anchor': {'identifier': 'anchor_959ec341-c694-4f7b-b040-258bfc191f2d',
  'sequence_number': 394},
 'id': 'page-21',
 'label': 'page',
 'resource_id': 'bosb002graa04_01'}

In [5]:
resource_id = page_annotation['resource_id']
begin = page_annotation['begin_anchor']['identifier']
end = page_annotation['end_anchor']['identifier']

response = requests.get(f"http://localhost:5000/{resource_id}/segmentedtext/textgrid/{begin},{end}")

In [6]:
response.json()

{'text_grid': {'_anchors': [{'identifier': 'anchor_5182d4c7-d319-4c85-a09c-fa58e6ba0958',
    'sequence_number': 386},
   {'identifier': 'anchor_e695cfd7-a4fb-40b5-81ca-e42292ff0dce',
    'sequence_number': 387},
   {'identifier': 'anchor_3dd5fa48-61f8-4de2-824b-28a73d64d102',
    'sequence_number': 388},
   {'identifier': 'anchor_5b5d42b6-ee90-4a35-a907-25a650b368f7',
    'sequence_number': 389},
   {'identifier': 'anchor_b1e0e773-e7f2-476c-8ebd-85c5159c7256',
    'sequence_number': 390},
   {'identifier': 'anchor_60f693f5-0f12-4714-b5b5-cbb465ad9b57',
    'sequence_number': 391},
   {'identifier': 'anchor_9467d8c0-5ef0-48bd-9480-08dd05e04af6',
    'sequence_number': 392},
   {'identifier': 'anchor_e841a860-301c-48cc-9df8-6975adccf9bd',
    'sequence_number': 393},
   {'identifier': 'anchor_959ec341-c694-4f7b-b040-258bfc191f2d',
    'sequence_number': 394},
   {'identifier': 'anchor_1c6253c6-c240-4497-9f4b-461df7bfc9d8',
    'sequence_number': 395}],
  '_ordered_segments': ['(de tegen

toon text_grid als een pseudo-visuele weergave, met om en om anchors en tussenliggende tekst, met de tekst geindenteerd.

In [7]:
# doorloop _anchors, toon anchor en segment bij die index
# textgrid = response.json()['text_grid']

textgrid = segmentedtext.SplittableSegmentedText.from_json(response.json()['text_grid'])

anchors = textgrid._anchors
segments = textgrid._ordered_segments

for i, segment in enumerate(segments):
    print(anchors[i].identifier+'\n')
    print(segment+'\n')
    
# and closing anchor
print(anchors[-1].identifier)

anchor_5182d4c7-d319-4c85-a09c-fa58e6ba0958

(de tegenspoeden vereenigen ware vrienden), want de mijne hebben mij allen verlaten; slechts u alleen, mijn getrouwste, konden zij niet afschrikken.’ En hem de hand reikende, sprak zij teeder: ‘Hoe goed, dat gij in den avond komt, maar weet ge wel dat het eene roekelooze vermetelheid is, dit heimelijk bezoek? Hoe zoude de koningin het opnemen als zij het eens vernam?’

anchor_e695cfd7-a4fb-40b5-81ca-e42292ff0dce

‘Zij zoude er zich zeker niet weinig over ergeren, dat ik hare verstooten zuster eerder mijne hulde durfde brengen dan haar zelve!’

anchor_3dd5fa48-61f8-4de2-824b-28a73d64d102

‘Hoe moet ik dit verstaan? Gij zijt immers reeds ten hove geweest?’

anchor_5b5d42b6-ee90-4a35-a907-25a650b368f7

‘Dat is het juist wat ik

anchor_b1e0e773-e7f2-476c-8ebd-85c5159c7256

niet

anchor_60f693f5-0f12-4714-b5b5-cbb465ad9b57

wilde. Het was immers uw wensch dat mijn eerste bezoek

anchor_9467d8c0-5ef0-48bd-9480-08dd05e04af6

u

anchor_e841a860-301c

Simuleer selectie van een stukje tekst, nl 'Whitehall' in het 1 na laatste segment.

In [8]:
begin_anchor_id = 'anchor_e841a860-301c-48cc-9df8-6975adccf9bd'
end_anchor_id = 'anchor_959ec341-c694-4f7b-b040-258bfc191f2d'
char_offset = 185
char_len = 9

print(segments[7])

gelden mocht? Gij ziet dien letterlijk vervuld, Mylady! Gisteren in den avond verliet ik den Tower, omhelsde vluchtig mijne zuster, liet door haren huisarts, dien ik vertrouwen kan, op Whitehall eene plotselinge ongesteldheid voorwenden, vermomde mij, steeg te paard en reed met Staunton hier heen - met die voorzorg evenwel, dat ik, toen het dag werd, een omweg door een paar dorpen nam om niemand op


In [9]:
print(segments[7][185:194])

Whitehall


In [11]:
# find begin and end anchors by id
begin_anchor = [a for a in anchors if a.identifier == begin_anchor_id][0]
end_anchor = [a for a in anchors if a.identifier == end_anchor_id][0]

print(f'ba: {begin_anchor} ea: {end_anchor}')

ba: anchor_e841a860-301c-48cc-9df8-6975adccf9bd ea: anchor_959ec341-c694-4f7b-b040-258bfc191f2d


In [12]:
entity_ann = {'resource_id': resource_id, 'label':'entity','begin_anchor': begin_anchor,\
              'end_anchor': end_anchor,'id': 'annot_'+str(uuid.uuid4()), 'entity_type': 'loc', \
              'entity_text': 'Whitehall'}

entity_ann

{'resource_id': 'bosb002graa04_01',
 'label': 'entity',
 'begin_anchor': anchor_e841a860-301c-48cc-9df8-6975adccf9bd,
 'end_anchor': anchor_959ec341-c694-4f7b-b040-258bfc191f2d,
 'id': 'annot_2dcaca12-4389-498a-abf5-143dbcbe1419',
 'entity_type': 'loc',
 'entity_text': 'Whitehall'}

In [13]:
entity_ann['char_offset'] = char_offset
entity_ann['char_len'] = char_len

entity_ann

{'resource_id': 'bosb002graa04_01',
 'label': 'entity',
 'begin_anchor': anchor_e841a860-301c-48cc-9df8-6975adccf9bd,
 'end_anchor': anchor_959ec341-c694-4f7b-b040-258bfc191f2d,
 'id': 'annot_2dcaca12-4389-498a-abf5-143dbcbe1419',
 'entity_type': 'loc',
 'entity_text': 'Whitehall',
 'char_offset': 185,
 'char_len': 9}

Sla de nieuwe annotatie op in annotationstore

In [14]:
headers = {"Content-Type": "application/json"}
url = "http://localhost:5001/annotations"
response = requests.put(url, data=json.dumps(entity_ann, indent=4, cls=segmentedtext.SegmentEncoder), headers=headers)

response

<Response [200]>

TODO NEXT: as a test retrieve annotations overlapping with page-21 and check if the newly created entity is included

In [15]:
url = "http://localhost:5001/bosb002graa04_01/annotations/anchor_5182d4c7-d319-4c85-a09c-fa58e6ba0958,anchor_959ec341-c694-4f7b-b040-258bfc191f2d"
response = requests.get(url)

In [16]:
response.json()

{'annotations': [{'begin_anchor': {'identifier': 'anchor_e695cfd7-a4fb-40b5-81ca-e42292ff0dce',
    'sequence_number': 387},
   'end_anchor': {'identifier': 'anchor_e695cfd7-a4fb-40b5-81ca-e42292ff0dce',
    'sequence_number': 387},
   'id': 'annot_c770527e-7562-49c5-9d03-06742e2f087e',
   'label': 'paragraph',
   'resource_id': 'bosb002graa04_01'},
  {'begin_anchor': {'identifier': 'anchor_3dd5fa48-61f8-4de2-824b-28a73d64d102',
    'sequence_number': 388},
   'end_anchor': {'identifier': 'anchor_3dd5fa48-61f8-4de2-824b-28a73d64d102',
    'sequence_number': 388},
   'id': 'annot_bb4c2950-82e6-40c8-a77a-2343b3d7cbca',
   'label': 'paragraph',
   'resource_id': 'bosb002graa04_01'},
  {'begin_anchor': {'identifier': 'anchor_5b5d42b6-ee90-4a35-a907-25a650b368f7',
    'sequence_number': 389},
   'end_anchor': {'identifier': 'anchor_959ec341-c694-4f7b-b040-258bfc191f2d',
    'sequence_number': 394},
   'id': 'annot_377db17d-cf7d-49ec-95b3-98fc934dfac7',
   'label': 'paragraph',
   'resource_